# Train a neural network for multi-label classification on the CelebA dataset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cleanlab/examples/blob/master/multilabel_classification/pytorch_network_training.ipynb)

This notebook demonstrates how to train a Pytorch neural network for image tagging and use the model to produce out-of-sample predicted class probabilities for each image in the dataset. These are required inputs to find label errors in multi-label classification datasets with cleanlab. Here we consider a subset of the [CelebA](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset, where each image may be tagged with one or more of the following tags: `['Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie', 'Eyeglasses', 'No_Beard', 'Smiling']`, depending on which ones apply to the person depicted.

This notebook only shows how to train the network and use it to produce `pred_probs`, using them to find label issues is demonstrated in our other [example](https://github.com/cleanlab/examples/) notebook on [Find Label Errors in Multi-Label Classification Data (CelebA Image Tagging)](https://github.com/cleanlab/examples/blob/multilabel_tutorial/multilabel_classification/image_tagging.ipynb). Here we fit a state-of-the-art neural network initialized from a pretrained [TIMM](https://timm.fast.ai/) network backbone. You can use this same code to obtain a multi-label classifier (i.e. image tagging model) for *any* image dataset.


Please install the dependencies specified in this [requirements.txt](https://github.com/cleanlab/examples/blob/master/multilabel_classification/requirements.txt) file before running the notebook. 

Next you need to download the dataset. The below code attempts to download the dataset from the Google Drive where its authors have stored it, but this Drive only allows limited programmatic downloads per day. If the Google Drive download script fails, please just manually download the following links from the [CelebA dataset webpage](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) and save them in the current working directory:
 * [Images](https://drive.google.com/uc?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM)
 * [Labels](https://drive.google.com/uc?id=0B7EVK8r0v71pblRyaVFSWGxPY0U)

In [61]:
# import gdown
# import os
# def download_drive(url,output):
#     filename = gdown.download(url, output, quiet=False)
#     if filename is None:
#         print(f"Downloading {url} failed, please download it from browser and paste it in {os.getcwd()}")
# url = 'https://drive.google.com/uc?id=0B7EVK8r0v71pblRyaVFSWGxPY0U'
# output = 'list_attr_celeba.txt'
# if not os.path.exists(output):
#     download_drive(url,output)
# url = 'https://drive.google.com/uc?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM' # Usually errors out, Download them manually from the link below
# output = 'img_align_celeba.zip'
# if not os.path.exists(output):
#     download_drive(url,output)

Remember you can just manually download the data from the link above if this code failed. Once you have downloaded the zipped data file, unzip the folder which contains a bunch of individual image files.

In [62]:
# !unzip -qq img_align_celeba.zip

Next we import the other required dependencies (make sure you have installed these packages).

In [1]:
import time
from collections import defaultdict
from collections import Counter
from types import SimpleNamespace

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import torch
from torch import nn
import torch.utils.data as data
from timm.optim import create_optimizer
from timm.models import create_model
from timm.data import resolve_data_config
from timm.data.loader import create_loader
from timm.utils import CheckpointSaver
from tqdm import tqdm

/home/alextay96/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alextay96/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/alextay96/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE
  warn(f"Failed to load image Python extension: {e}")


Now we load the dataset, and preprocess it to only keep the classes of interest (i.e. *image tags*) listed above.

In [64]:
DATA =("image_id " + open("list_attr_celeba.txt").read()[7:]).splitlines()

from collections import defaultdict
q = DATA[0]
d_data = defaultdict(list)
ls = q.split()
for i in q.split():
    d_data[i] = []
for j in DATA[1:]:
    labels = j.split()
    for k in range(0,len(labels)):
        if k==0:
            d_data[ls[k]].append(labels[k])
        else:
            # map -1 entries -> 0
            ps = int((int(labels[k])+1)/2)
            d_data[ls[k]].append(ps)
            
dat = pd.DataFrame.from_dict(d_data)


selected = ['image_id',
'Eyeglasses',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'No_Beard',
 'Smiling']

def is_label(row):
    for s in selected[1:]:
        if row[s]!=0:
            return True
    return False

def get_loc(i):
    return os.path.join(os.getcwd(),'img_align_celeba/')+i

dat_label = dat.apply(is_label,axis=1)
dat_selected = dat[dat_label][selected]
dat_selected['image_path'] = dat_selected['image_id'].map(lambda x:get_loc(x))
selected[0] = 'image_path'

df = dat_selected[selected]
set_lab = {}
for i,row in df.iterrows():
    q = str(row.tolist()[1:])
    if q not in set_lab:
        set_lab[(str(q))]=len(set_lab)

# Here we drop a couple rare class-combinations just to simplify stratified data splitting
def get_lab(row):
    q = str(row.tolist()[1:])
    return set_lab[q]

df['unique_label'] = df.apply(get_lab,axis=1)
cnt = Counter(df['unique_label'])

def drop(val):
    if cnt[val]>10:
        return True
    return False

is_drop = df['unique_label'].apply(lambda x:drop(x))
df = df[is_drop]

The resulting DataFrame contains the ID of each image and the classes (i.e. *tags*) that apply to this image.

We define a general class of Pytorch neural networks for multi-label image classification. You can use this class for training models on *any* image dataset, and this class can utilize *any* pretrained [TIMM](https://timm.fast.ai/) network backbone. The `MultiLabelModel` below adds an appropriate output layer on top of the network backbone and then fine-tunes the entire network jointly on your multi-label classification dataset.

In [65]:
class MultiLabelModel(nn.Module):
    """ 
    Pytorch network for multi-label classification that can utilize any TIMM network backbone.
    Some of this code is inspired by: https://github.com/yang-ruixin/PyTorch-Image-Models-Multi-Label-Classification
    Note this network uses Sigmoid output activations because the predicted probabilities do not need to sum to 1
    for multi-label classification, in which each image may belong to multiple classes rather than only one.
    """
    def __init__(self, model, n_classes, class_weights=None, verbose = False):
        super().__init__()
        self.base_model = model  # network backbone can be any TIMM model
        self.num_classes = n_classes
        self.verbose = verbose

    def forward(self, x):
        x = self.base_model(x)

        x = torch.flatten(x, 1)

        return x

    def get_loss(self, loss_fn, output, target):

        return loss_fn(output, target)

    def validate(self, loader):
        self.eval();
        with torch.no_grad():
            total_loss = 0
            m = nn.Sigmoid()
            labels = []
            preds = []
            for batch_idx, (input, target) in enumerate(loader):
                input = input.cuda()
                labels.append(target.detach().cpu())
                target = target.float().cuda()
                output = m(self(input))
                loss = self.get_loss(loss_fn, output, target)

                total_loss += loss.item()
                pred_model = (output > 0.5).detach().cpu()
                preds.append(pred_model)

            num_of_batches_per_epoch = len(loader)
            avg_loss = total_loss / num_of_batches_per_epoch
            print("VALIDATION DATA STATS")

            print("AVERAGE LOSS:", avg_loss)
            preds = torch.cat(preds).int()
            labels = torch.cat(labels).int()
            acc_score = accuracy_score(labels, preds)
            print("MULTILABEL accuracy score:", acc_score)
            per_class = []
            for i in range(len(preds.T)):
                per_class.append(accuracy_score(labels.T[i], preds.T[i]))
            print(dataset_train.label_names)
            print(per_class)
            print('\n\n')
        return avg_loss

    def predict_proba(self, loader):
        self.eval();
        with torch.no_grad():
            m = nn.Sigmoid()
            preds = []
            for batch_idx, (input, target) in enumerate(loader):
                input = input.cuda()
                output = m(self(input))
                pred_model = output.detach().cpu()
                preds.append(pred_model)
            preds = torch.cat(preds)
        return preds

    def train_one_epoch(
        self,
        loader,
        optimizer,
        loss_fn,
    ):
        sta = time.time()
        second_order = hasattr(optimizer, "is_second_order") and optimizer.is_second_order
        self.train()
        total_loss = 0
        m = nn.Sigmoid()
        labels = []
        preds = []
        ct = 0
        for batch_idx, (input, target) in tqdm(enumerate(loader)):
            input = input.cuda()
            ct += 1
            labels.append(target.detach().cpu())
            target = target.float().cuda()
            output = m(self(input))
            loss = self.get_loss(loss_fn, output, target)
            total_loss += loss.item()
            pred_model = (output > 0.5).detach().cpu()
            preds.append(pred_model)
            optimizer.zero_grad()
            loss.backward(create_graph=second_order)
            optimizer.step()
            if ct % 80 == 0 and self.verbose:
                print("LOSS:", loss.item())
        num_of_batches_per_epoch = len(loader)
        avg_loss = total_loss / num_of_batches_per_epoch
        print("TRAINING DATA STATS")
        print("AVERAGE LOSS:", avg_loss)
        preds = torch.cat(preds).int()
        labels = torch.cat(labels).int()
        acc_score = accuracy_score(labels, preds)
        print("MULTILABEL accuracy score:", acc_score)
        per_class = []
        for i in range(len(preds.T)):
            per_class.append(accuracy_score(labels.T[i], preds.T[i]))
        print(dataset_train.label_names)
        print(per_class)
        print('\n\n')
        sto = time.time()
        print("training time", sto - sta)
        return avg_loss
    

    def fit(self, loader_train, load_val, num_epochs=10):
        if os.path.exists("weights_model"):
            print("removing weights directory")
            os.system('rm -rf weights_model')
        os.mkdir("weights_model")
        args = SimpleNamespace()
        args.weight_decay = 0
        args.lr = 1e-4
        args.opt = 'adam'
        args.momentum = 0.9
        args.sched = "step"

        optimizer = create_optimizer(args, self)
        saver = CheckpointSaver(
            model=self,
            optimizer=optimizer,
            checkpoint_dir="weights_model"
        )
        errs = []
        num_of_data_train = len(loader_train.dataset.data)
        for epoch in range(0, num_epochs):
            loss_train = self.train_one_epoch(
                loader_train,
                optimizer,
                loss_fn,
            )
            loss_val = self.validate(loader_val)
            errs.append([loss_train, loss_val])
            saver.save_checkpoint(epoch, metric=loss_val)

We also create a wrapper class for training our `MultiLabel` model on multi-label classification image datasets. You can easily apply this to your own datasets.

In [66]:
class DatasetMultiLabel(data.Dataset):
    def __init__(
            self,
            annotation_path=None,
            df = None,
            transform=None):

        super().__init__()
        self.transform = transform
        self.data = []
        self.labels = []
        self.label_names = []
        if annotation_path is None:
            assert df is not None
        else:
            df = pd.read_csv(annotation_path)
        
        cols = df.columns
        self.label_names = list(cols[1:-1])
        for i,row in df.iterrows():
            lb = []
            for j in cols:
                if j=='unique_label':
                    continue
                if j=='image_path':
                    self.data.append(row[j])
                else:
                    lb.append(float(row[j]))
            self.labels.append(lb)
                
    def __getitem__(self, idx):
        img_path = self.data[idx]

        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)

        labels = self.labels[idx]

        return img, labels

    def __len__(self):
        return len(self.data)

In [67]:
def reset_weights(m):
  '''
  Re-initializes model weights, eg. between cross-validation folds. 
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

def create_df(pred_probs, dataset):
    """
    Returns a dataframe with image_path and predicted probabilities for each image.
    """
    ls = dataset_val.label_names
    cl = defaultdict(list)
    cl['image_path'] = dataset.data
    for i in range(0,len(ls)):
        cl[ls[i]] = pred_val.T[i].tolist()
    return pd.DataFrame.from_dict(cl)

Let's create a `DatasetMultiLabel` and `MultiLabelModel` for the Celeb-A dataset. Here we use the [efficientnet_b0](https://rwightman.github.io/pytorch-image-models/models/tf-efficientnet/) backbone for our neural network, but you can easily use any other TIMM backbone. 

In [68]:
dataset = DatasetMultiLabel(df = df)

loss_fn = nn.BCELoss()

model = create_model(
    'efficientnet_b0',  # you can replace this with any TIMM backbone
    num_classes=len(dataset.labels[0]),
)
data_config = resolve_data_config(
       args = {}, model=model
    )
data_config["input_size"] = (3, 32, 32)
model = MultiLabelModel(
        model,
        n_classes=len(dataset.labels[0]),
    ).cuda()

We train this network using K-fold cross validation. This allows us to obtain **out-of-sample** predictions for each image in the dataset (i.e. predictions from a copy of the model which never saw this image during training). Out-of-sample predictions are less prone to overfitting, and thus better suited for finding label issues. From each fold of cross-validation, we store the predicted class probabilities for the images that were out-of-sample in a DataFrame `df_pred`. These predictions are subsequently used for finding label issues in cleanlab's Tutorial on [Multi-Label Classification](https://docs.cleanlab.ai/).

In [3]:
num_splits = 2  # number of cross-validation splits (higher values will take longer but give better results)
skf = StratifiedKFold(n_splits=num_splits)

In [70]:
ct = 1
for train_index, test_index in skf.split(df,df['unique_label']):
    if ct!=1:
        model.apply(reset_weights);
    dataset_train = DatasetMultiLabel(df = df.iloc[train_index])
    dataset_val = DatasetMultiLabel(df = df.iloc[test_index])
    loader_train = create_loader(
        dataset_train,
        input_size=data_config["input_size"],
        batch_size=500,
        is_training=True,
        mean=data_config["mean"],
        std=data_config["std"],
       interpolation=data_config["interpolation"],
    )
    loader_val = create_loader(
        dataset_val,
        input_size=data_config["input_size"],
        batch_size=500,
        is_training=False,
        mean=data_config["mean"],
        std=data_config["std"],
        interpolation=data_config["interpolation"],

    )
    model.fit(loader_train,loader_val,num_epochs=1)
    checkpoint = torch.load("weights_model/model_best.pth.tar")
    model.load_state_dict(checkpoint['state_dict'])
    pred_val = model.predict_proba(loader_val)
    df_pred = create_df(pred_val,dataset_val)
    df_pred.to_csv(str(ct)+"_fold.csv",index=False)
    ct+=1

removing weights directory


188it [00:48,  3.87it/s]

TRAINING DATA STATS
AVERAGE LOSS: 1.3429630997967212
MULTILABEL accuracy score: 0.1914787234042553
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie', 'No_Beard', 'Smiling']
[0.8731489361702127, 0.7407446808510638, 0.8840957446808511, 0.7922234042553191, 0.8586702127659575, 0.782563829787234, 0.5045851063829787]



training time 48.6141881942749


VALIDATION DATA STATS
AVERAGE LOSS: 0.4749027927401205
MULTILABEL accuracy score: 0.26598468189978436
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie', 'No_Beard', 'Smiling']
[0.916972072617568, 0.7926426378575905, 0.9459192453551738, 0.8649521442153457, 0.9135515259674729, 0.8928689038316496, 0.5250751564209609]



removing weights directory


188it [00:49,  3.81it/s]

TRAINING DATA STATS
AVERAGE LOSS: 0.4306276840098361
MULTILABEL accuracy score: 0.2398191489361702
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie', 'No_Beard', 'Smiling']
[0.9177872340425532, 0.7653191489361703, 0.9204893617021277, 0.853095744680851, 0.9091170212765958, 0.8769255319148936, 0.5245]



training time 49.3691520690918


VALIDATION DATA STATS
AVERAGE LOSS: 0.3744510628243603
MULTILABEL accuracy score: 0.2624288263788561
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie', 'No_Beard', 'Smiling']
[0.9302923429931164, 0.7971551797399508, 0.9481069941361434, 0.8679569983853148, 0.9220383275261324, 0.8982748364069006, 0.5427891561145577]





In [4]:
dfl = []
for i in range(1,num_splits+1):
    dfl.append(pd.read_csv(str(i)+"_fold.csv"))

cols = dfl[0].columns[1:]
df_pred = pd.concat(dfl,axis=0)
df_pred['image_path'] = (df_pred['image_path'].map(lambda x:x.split('/')[-1]))
df_pred.set_index('image_path').to_csv("pred_probs.csv")

In [5]:
df_pred

,image_path,Eyeglasses,Wearing_Earrings,Wearing_Hat,Wearing_Necklace,Wearing_Necktie,No_Beard,Smiling
0,000001.jpg,0.213967,0.230944,0.120494,0.112896,0.297117,0.810098,0.410487
1,000002.jpg,0.175427,0.222502,0.099109,0.203868,0.165654,0.844859,0.402193
2,000003.jpg,0.121280,0.216154,0.076970,0.147503,0.115196,0.903464,0.486187
3,000004.jpg,0.146671,0.226989,0.082455,0.169462,0.133613,0.884957,0.497215
4,000005.jpg,0.181739,0.251470,0.066860,0.170052,0.139145,0.857879,0.526083
...,...,...,...,...,...,...,...,...
94131,202595.jpg,0.057444,0.217195,0.048693,0.142870,0.067489,0.917330,0.522584
94132,202596.jpg,0.048415,0.224178,0.038581,0.144111,0.060461,0.915048,0.552987
94133,202597.jpg,0.123990,0.228380,0.130657,0.126525,0.097333,0.861600,0.421455
94134,202598.jpg,0.078667,0.159727,0.078418,0.110420,0.070555,0.886762,0.564058


In [6]:


import numpy as np
import os
import torchvision
import matplotlib.pyplot as plt
import pandas as pd

from cleanlab.filter import find_label_issues
from cleanlab.internal.multilabel_utils import onehot2int

np.set_printoptions(suppress=True)

In [7]:
df_pred = pd.read_csv("pred_probs.csv").set_index('image_path').sort_index()
DATA =("image_path " + open("list_attr_celeba.txt").read()[7:]).splitlines()

from collections import defaultdict
q = DATA[0]
d_data = defaultdict(list)
ls = q.split()
for i in q.split():
    d_data[i] = []
for j in DATA[1:]:
    labels = j.split()
    for k in range(0,len(labels)):
        if k==0:
            d_data[ls[k]].append(labels[k])
        else:
            ps = int((int(labels[k])+1)/2)
            d_data[ls[k]].append(ps)

df = pd.DataFrame.from_dict(d_data).set_index("image_path").loc[df_pred.index].sort_index()
class_names = df_pred.columns
binarized_labels = df[class_names].to_numpy()
pred_probs = df_pred[class_names].to_numpy()

In [8]:
def get_label_name(ls):
    return [class_names[i] for i in ls]

def get_image_loc(s):
    return os.getcwd() + "/img_align_celeba/" + s

In [14]:
binarized_labels

array([[0, 1, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 1, 1],
       [0, 1, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [9]:
labels = onehot2int(binarized_labels)  # label for each image in the format required by cleanlab (integer class indices)
pred_labels = onehot2int(np.array((pred_probs > 0.5), np.int32))  # predicted classes to which each image belongs, according to our model
image_names = np.array([get_image_loc(i) for i in df.index])  # name of file for each image
pred_label_names = [get_label_name(i) for i in pred_labels]  # class predictions in terms of original class names
label_names = [get_label_name(i) for i in labels]  # image labels in terms of original class names

In [15]:
labels

[[1, 5, 6],
 [5, 6],
 [5],
 [1, 3, 5],
 [5],
 [1, 5],
 [5],
 [5],
 [1, 5, 6],
 [5],
 [5, 6],
 [5, 6],
 [5, 6],
 [3, 5, 6],
 [4, 5],
 [6],
 [5, 6],
 [1, 3, 5, 6],
 [3, 5],
 [4, 5, 6],
 [3, 5],
 [5, 6],
 [1, 3, 5],
 [5],
 [5, 6],
 [5, 6],
 [1, 5, 6],
 [5],
 [5, 6],
 [4, 5],
 [5, 6],
 [5],
 [5],
 [5],
 [2],
 [5, 6],
 [1, 5, 6],
 [1, 5],
 [6],
 [1, 5, 6],
 [1, 5, 6],
 [3, 5],
 [1, 5, 6],
 [5, 6],
 [1, 5, 6],
 [5],
 [5, 6],
 [5],
 [0],
 [3, 5, 6],
 [4, 5],
 [5],
 [1, 5],
 [5],
 [5],
 [5],
 [3, 5, 6],
 [5, 6],
 [6],
 [5, 6],
 [1, 3, 5, 6],
 [5],
 [2, 5, 6],
 [5],
 [5, 6],
 [1, 5, 6],
 [4, 5, 6],
 [5],
 [1, 2, 5],
 [5],
 [6],
 [5, 6],
 [5],
 [5, 6],
 [4, 5],
 [5, 6],
 [6],
 [1, 3, 5, 6],
 [1, 5],
 [1, 3, 5],
 [3, 5],
 [3, 5, 6],
 [5],
 [5],
 [5],
 [4, 5],
 [5, 6],
 [0, 3, 5],
 [3, 5, 6],
 [1, 2, 5],
 [5],
 [5],
 [5],
 [5, 6],
 [5],
 [5],
 [3, 5],
 [4, 5, 6],
 [5],
 [1, 3, 5, 6],
 [3, 5, 6],
 [1, 5, 6],
 [4],
 [3, 5, 6],
 [1, 5, 6],
 [5, 6],
 [5, 6],
 [5, 6],
 [5],
 [4, 5, 6],
 [1, 5],
 [5, 6]

In [16]:
type(pred_labels)

list

In [10]:
num_to_display = 3  # increase this to see more examples

print(f"labels for first {num_to_display} examples in cleanlab format:")
print(labels[:num_to_display])
print(f"pred_probs for first {num_to_display} examples in cleanlab format:")
print(pred_probs[:num_to_display])

labels for first 3 examples in cleanlab format:
[[1, 5, 6], [5, 6], [5]]
pred_probs for first 3 examples in cleanlab format:
[[0.21396729 0.23094364 0.12049425 0.11289628 0.29711702 0.81009769
  0.41048697]
 [0.17542657 0.22250192 0.09910902 0.2038683  0.16565426 0.8448593
  0.40219277]
 [0.12127966 0.21615355 0.07696974 0.14750262 0.11519588 0.90346438
  0.4861871 ]]


In [11]:
def plot_image(pos, pred_label=None):
    """ Plots particular CelebA image and its label, and optionally the model-predicted label as well. """
    label_error = label_names[pos]
    path = image_names[pos]
    img = torchvision.io.read_image(path).swapaxes(0, 2).swapaxes(0, 1)
    plt.axis("off")
    plt.imshow(img)
    fontsize = 20
    plt.text(
        200,
        30,
        image_names[pos].split("/")[-1],
        bbox=dict(
            fill=False,
            edgecolor="black",
            linewidth=1,
        ),
        fontsize=fontsize,
    )
    txt = plt.text(
        200,
        120,
        "Given Label: " + str(label_error),
        bbox=dict(fill=False, edgecolor="red", linewidth=1),
        fontsize=fontsize,
        wrap=True,
    )
    txt._get_wrap_line_width = lambda: 400
    if pred_label is not None:
        txt = plt.text(
            200,
            200,
            "Model Predicted: " + str(pred_label),
            bbox=dict(fill=False, edgecolor="green", linewidth=1),
            fontsize=fontsize,
            wrap=True,
        )
        txt._get_wrap_line_width = lambda: 400

In [12]:
example_to_view = 0  # you can play with this
plot_image(example_to_view)

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'